<a href="https://colab.research.google.com/github/pachterlab/seqspec/blob/libspec/examples/seqspec-dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf seqspec
!git clone https://github.com/pachterlab/seqspec
!cd seqspec && git checkout -b libspec origin/libspec && pip install --quiet .

Cloning into 'seqspec'...
remote: Enumerating objects: 1541, done.
remote: Counting objects: 100% (719/719), done.
remote: Compressing objects: 100% (382/382), done.
remote: Total 1541 (delta 446), reused 557 (delta 331), pack-reused 822
Receiving objects: 100% (1541/1541), 6.15 MiB | 15.04 MiB/s, done.
Resolving deltas: 100% (967/967), done.
Branch 'libspec' set up to track remote branch 'libspec' from 'origin'.
Switched to a new branch 'libspec'
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.8 MB/s eta 0:00:00


In [2]:
!seqspec

usage: seqspec [-h] <CMD> ...

seqspec 0.1.1: Format sequence specification files

positional arguments:
  <CMD>
    check     validate seqspec file
    find      find regions in a seqspec file
    format    format seqspec file
    genbank   get genbank about seqspec file
    index     index regions in a seqspec file
    info      get info about seqspec file
    init      init a seqspec file
    modify    modify region attributes
    onlist    get onlist file for specific regions
    print     print seqspec file
    split     split seqspec into modalities
    version   Get seqspec version and seqspec file version

options:
  -h, --help  show this help message and exit


In [3]:
!head seqspec/docs/assays/illumina_truseq_dual.spec.yaml

!Assay
seqspec_version: 0.0.0
assay_id: Illumina-novaseq-truseq-dual-index
name: Example assay
doi: https://support.illumina.com/content/dam/illumina-support/documents/documentation/system_documentation/miseq/indexed-sequencing-overview-guide-15057455-08.pdf
date: 21 February 2024
description: Example seqspec for an assay
modalities:
- rna
lib_struct: null


In [4]:
!seqspec print seqspec/docs/assays/illumina_truseq_dual.spec.yaml

                                        ┌─'illumina_p5:29'
                                        ├─'truseq_read1:10'
                                        ├─'insert:150'
─────────────────── ──rna───────────────┤
                                        ├─'truseq_read2:34'
                                        ├─'index7:8'
                                        └─'illumina_p7:24'


In [5]:
!seqspec check seqspec/docs/assays/illumina_truseq_dual.spec.yaml

[error 1] None is not of type 'string' in spec['lib_struct']
[error 2] 'insert' is not one of ['atac', 'barcode', 'cdna', 'crispr', 'custom_primer', 'dna', 'fastq', 'fastq_link', 'gdna', 'hic', 'illumina_p5', 'illumina_p7', 'index5', 'index7', 'linker', 'ME1', 'ME2', 'methyl', 'named', 'nextera_read1', 'nextera_read2', 'poly_A', 'poly_G', 'poly_T', 'poly_C', 'protein', 'rna', 's5', 's7', 'tag', 'truseq_read1', 'truseq_read2', 'umi'] in spec['library_spec'][0]['regions'][2]['region_type']
[error 3] None is not of type 'string' in spec['library_spec'][0]['regions'][4]['onlist']['md5']
[error 3] index7_onlist.txt does not exist
[error 4] R1.fastq.gz file does not exist
[error 5] I1.fastq.gz file does not exist
[error 6] I2.fastq.gz file does not exist
[error 7] R2.fastq.gz file does not exist


In [6]:
!seqspec index -m rna -r R1.fastq.gz seqspec/docs/assays/illumina_truseq_dual.spec.yaml

R1.fastq.gz	insert	0	98


In [7]:
!seqspec index --rev -m rna -r R2.fastq.gz seqspec/docs/assays/illumina_truseq_dual.spec.yaml

R2.fastq.gz	insert	0	98


In [8]:
!seqspec find -m rna -r insert seqspec/docs/assays/illumina_truseq_dual.spec.yaml

- !Region
  region_id: insert
  region_type: insert
  name: Custom insert
  sequence_type: random
  sequence: X
  min_len: 1
  max_len: 150
  onlist: null
  regions: null
  parent_id: rna



# Code testing

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.dates as mdates

from seqspec.utils import load_spec, get_cuts
from seqspec.seqspec_index import run_index
from seqspec.seqspec_find import run_find
import os

fsize=15

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

In [13]:
specA = load_spec("seqspec/docs/assays/element_adept_truseq_dual.spec.yaml")
specB = load_spec("seqspec/docs/assays/illumina_truseq_dual.spec.yaml")

# modality
mA = "rna"
mB = "rna"

modeA = specA.get_libspec(mA)
modeB = specB.get_libspec(mA)

# Get the regions from specA based on the FASTQ file names
fqA_fns = ["data/R1.fastq.gz", "data/R2.fastq.gz"]
fqB_fns = ["data/R1.fastq.gz", "data/R2.fastq.gz"]



In [14]:
print(modeA.sequence)

AATGATACGGCGACCACCGAGATCTACACTCTTTCCCTACACGACGCTCTTCCGATCTXAGATCGGAAGAGCACACGTCTGAACTCCAGTCACNNNNNNNNATCTCGTATGCCGTCTTCTGCTTG


In [22]:
# atac
# methyl
# rna
# protein
# hic
# crispr

colors = {
"barcode": '#2980B9',
"cdna": '#8E44AD',
"fastq": '#F1C40F',
"gdna": '#E67E22',
"illumina_p5": '#E17A47',
"illumina_p7": '#E17A47',
"index5": '#4AB19D',
"index7": '#4AB19D',
"linker": '#1ABC9C',
"ME1": '#E74C3C',
"ME2": '#E74C3C',
"nextera_read1": '#FF8000',
"nextera_read2": '#FF8000',
"poly_A": '#FF0000',
"poly_G": '#C0C0C0',
"poly_T": '#7F8C8D',
"poly_C": '#2C3E50',
"s5": '#EF3D59',
"s7": '#EF3D59',
"truseq_read1": '#EFC958',
"truseq_read2": '#EFC958',
"umi": '#16A085',
"tag": '#344E5C',
"protein": '#ECF0F1',
           "insert": '#FF8C00'}




'#95A5A6'

'#95A5A6'

In [17]:
spec = load_spec("seqspec/docs/assays/illumina_truseq_dual.spec.yaml")

# modality
modalities = spec.list_modalities()
modes = [spec.get_libspec(m) for m in modalities]
lengths = [i.min_len for i in modes]
nmodes = len(modalities)

In [18]:
# sort the modalities by their lengths
asort = np.argsort(lengths)
modalities = np.array(modalities)[asort].tolist()
lengths = np.array(lengths)[asort].tolist()
modes = np.array(modes)[asort].tolist()